## Première régression linéaire 

On a dit que choisir un modèle c'était se restreindre la famille de fonction qu'on cherche. 

Si on choisit d'utiliser une régression linéaire, on choisit une famille de fonction de la forme 
$ \hat{y} = \theta_0 + \theta_1 * X_1 + \theta_2 * X_2 + ... + \theta_n * X_n $ 

Le but est alors de trouver les bonnes valeurs de $\theta$ pour que les prédictions du modèles sont le mieux possible. 

Scikit-learn nous permet de trouver les valeurs automatiquement

![ml_machine_learning.png](ml_machine_learning.png)

In [ ]:
#Prenons un dataset synthétique
import numpy as np

X = np.abs(2.2 + np.random.randn(20, 1))  #année experience pour 20 personnes différentes
y = 500 + 900 * X 

In [ ]:
y[:5] # salaire 

array([[2518.3124357 ],
       [2010.23942191],
       [2021.24713798],
       [2485.03587887],
       [1659.25243561]])

In [ ]:
# comment établir autoamtique la relation entre X et y , cad trouver les theta

from sklearn.linear_model import LinearRegression
 # theta0 + theta1 * X1 + theta2 * X2 + ....

model = LinearRegression() # création de la fonction
model.fit(X, y) #recherche des valeurs de theta automatiquement.

print(model.intercept_) # theta0
print(model.coef_) #theta1

[500.]
[[900.]]


In [ ]:
model.predict([[20]]) #prédit moi mon salaire si j'avais 20 ans experiences

array([[18500.]])

In [ ]:
import pandas as pd


df = pd.DataFrame({"size": [100, 200, 50], "orientation": ['Sud', 'Nord', 'Est'], "prix": [1111, 3000, 500]})
df

,size,orientation,prix
0,100,Sud,1111
1,200,Nord,3000
2,50,Est,500


In [ ]:
#la colonne orientation est au format "text"
# on est pas content il faut le représenter sous forme numérique

# il y a deux options :
# Utiliser un encodage catégoriel (Sud -> 0, Nord -> 1, Est -> 2 )
# Utiliser un encodage "one hot encoding"

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(df['orientation'])


result = encoder.transform(df['orientation'])
result


array([2, 1, 0])

In [ ]:
df['orientation'] = result
df

,size,orientation,prix
0,100,2,1111
1,200,1,3000
2,50,0,500


In [ ]:
df = pd.DataFrame({"size": [100, 200, 50, 50, 50], "orientation": ['Sud', 'Nord', 'Sud Est', 'Est', 'Ouest'], "prix": [1111, 3000, 500, 600, 520]})
df

,size,orientation,prix
0,100,Sud,1111
1,200,Nord,3000
2,50,Sud Est,500
3,50,Est,600
4,50,Ouest,520


In [ ]:
df = pd.get_dummies(df, prefix=['orientation']) #transforme moi la colonne orientation en one hot encoding (aussi dummies)
df

,size,prix,orientation_Est,orientation_Nord,orientation_Ouest,orientation_Sud,orientation_Sud Est
0,100,1111,0,0,0,1,0
1,200,3000,0,1,0,0,0
2,50,500,0,0,0,0,1
3,50,600,1,0,0,0,0
4,50,520,0,0,1,0,0


mon modèle a plus de variable : au lieu d'une seule information orientation, on va mettre une variable par direction

$ \hat{y} = prix = \theta_0 + \theta_1 * size + \theta_2 * est? + \theta_3 * nord ? + \theta_4 * Ouest? + \theta_5 * Sud ? $  

si la maison est au sud
et qu'elle fait 100 en taille

on $ \hat{y} = \theta_0 + \theta_1 * size + \theta_2 * 0 + \theta_3 * 0 + \theta_4 * 0 + \theta_5 * 1$


en faisant ça on découple chaque orientation pour qu'il n'y ait pas de relation d'ordre

In [ ]:
df

,size,prix,orientation_Est,orientation_Nord,orientation_Ouest,orientation_Sud,orientation_Sud Est
0,100,1111,0,0,0,1,0
1,200,3000,0,1,0,0,0
2,50,500,0,0,0,0,1
3,50,600,1,0,0,0,0
4,50,520,0,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(df)



In [ ]:
df_train

,size,prix,orientation_Est,orientation_Nord,orientation_Ouest,orientation_Sud,orientation_Sud Est
1,200,3000,0,1,0,0,0
0,100,1111,0,0,0,1,0
2,50,500,0,0,0,0,1


In [ ]:
df_test

,size,prix,orientation_Est,orientation_Nord,orientation_Ouest,orientation_Sud,orientation_Sud Est
4,50,520,0,0,1,0,0
3,50,600,1,0,0,0,0


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

X_train = df_train[['size', 'orientation_Est', 'orientation_Ouest', 'orientation_Nord', 'orientation_Sud']]
y_train = df_train['prix']

model.fit(X_train, y_train)

LinearRegression()

In [ ]:
#le modèle a été entraîner on va l'évaluer sur le train et sur le test ! !!! 
from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(model.predict(X_train), y_train)

X_test = df_test[['size', 'orientation_Est', 'orientation_Ouest', 'orientation_Nord', 'orientation_Sud']]
y_test = df_test['prix']


#nulle part j'ai fais de fit sur x_test, y_test donc on n'a jamais révélé les réponses du test au modèles
mse_test = mean_squared_error(model.predict(X_test), y_test)

In [ ]:
mse_train, mse_test #bon score sur le train, mauvais score sur le test : généralise pas (apprit par coeur : overfitting :())

(4.006656092053728e-25, 5199.999999999933)

In [ ]:
# pour lutter l'apprentissage par coeur il y a plusieurs méthode
# la première consiste limité la capacité du modèle (la taille du cerveau)
# la seconde :  plus de donnée d'entraînement => plus dur d'apprendre par coeur




In [ ]:
# dans sklearn il existe d'autres modèles de régression 

#chaque modèle a sa propre formule mathématique 
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()

model.fit(X_train, y_train)

#etc

DecisionTreeRegressor()

In [ ]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge()
model.fit(X_train, y_train)

BayesianRidge()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
# mais si on veut faire de la classification ? 


#Le plus simple :

from sklearn.linear_model import LogisticRegression # C4EST PAS PARCE QUE Y A MARQUE REGRESSION QUE CEST DE LA REGRESSION : CEST DE LA CLASSIFICATION


#classification on prédit une variable discrète : 0, 1, 2 , Ou 3 (chat , chien, girafe, poulet)



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25ad01f7-ec4b-4ac8-bce6-a3745ef02c96' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>